### Initial Data Load and Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
! pip install -q kaggle
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/Data/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c aptos2019-blindness-detection
! unzip aptos2019-blindness-detection

### Imported Libraries

In [3]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import *
from keras import layers
from keras.models import Sequential, Model
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.applications import InceptionV3
from keras.models import load_model
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import shutil
import cv2
import os

### Function to create folders and split data

In [43]:
def splitter(data, label):
  # Create Folders
  try:
    os.mkdir(label)
  except FileExistsError:
    pass
  names = ["NoDR", "Mild", "Moderate", "Severe", "ProliferativeDR"]
  for i in range(len(names)):
    try:
      os.mkdir(os.path.join(label, names[i]))
    except FileExistsError:
      pass

  # Splits Data
  identifier = {}
  # Sorts Data
  trainSortedData = data[data[:, 1].argsort()]
  # Identifies the unique value and stoes into identifier
  for i in np.unique(trainSortedData[:, 1]):
    identifier[i] = trainSortedData[trainSortedData[:, 1] == i]
  # Creates variable and stores names and data into its respective variables
    for i in identifier:
        name = names[int(i)]
        array = identifier[i]
  # Identifies and moves the images to respective folders
        for i in array:
            x = str(i[0]) + ".png"
            srcPath = os.path.join("train_images", x)
            dstPath = os.path.join(label, name, x)
            try:
              shutil.move(srcPath, dstPath)
            except FileNotFoundError:
              pass


### Predict Function

In [44]:
def predict(model, img_path):
    # Load image and preprocess it
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.

    # Make prediction
    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction)
    class_labels = {0: 'NoDR', 1: 'Mild', 2: 'Moderate', 3: 'Severe', 4: 'ProliferativeDR'}
    predicted_class = class_labels[class_idx]

    return predicted_class

### Splitting Dataset

In [ ]:
# Initial Dataset
try:
  initialData = pd.read_csv("train.csv")
  initialData = np.array(initialData)
  usingData = initialData[:3600]
except FileNotFoundError:
  pass
# Split to Alien Dataset from Initial Dataset(last 62 rows)
alienData = np.array(initialData[3600:])
alienFolder = splitter(alienData, "alien")

# Split to Train Dataset from Initial Dataset(80%)
trainData = usingData[:(int)(len(usingData)*0.80), :]
trainFolder = splitter(trainData, "train")

# Split to Validation Dataset from Initial Dataset(20%)
validationData = usingData[(int)(len(usingData)*0.80):, :]
validationFolder = splitter(validationData, "validation")

### Remove unused folders and data

In [ ]:

! rm -rf sample_data/ test_images/ train_images/ aptos2019-blindness-detection.zip sample_submission.csv test.csv train.csv

### Moving Alien Dataset from the working directory

In [ ]:
try:
  shutil.move('/content/alien', '/content/drive/MyDrive/Model')
except FileNotFoundError:
  pass

### Parameters for Data Augmentation and Model

In [ ]:
height = 224 # Image Height
width = 224 # Image Width

### Data Augmentation on train folder

In [ ]:
train_datagen = ImageDataGenerator (
                                    rescale = 1/255, # Rescaling images 0 to 1 also known as normalizing
                                    rotation_range = 10, # Range from 0 to 5 degress rotation (Can be both ways)
                                    zoom_range = 0.10, # Range from 0% to 5% to either Zoom In or Zoom Out
                                    horizontal_flip = True, # Horizontal flip needed not vertical
                                    
                                   )
train_generator = train_datagen.flow_from_directory (
                                                     '/content/train',
                                                     batch_size = 16, # Batch passing
                                                     class_mode = 'categorical', # categorical since multiple classes
                                                     target_size = (
                                                                    height, width # Resize
                                                                   )
                                                    )

### Generating Data Augmentation for display

In [ ]:
# Generating Data Augmentation for display
images, labels = next(train_generator) # iteratting

# Plot in matrix
fig, axes = plt.subplots(1, 4, figsize=(16, 8))
axes = axes.ravel()

# Generating 5 images
for i in range(4):
    axes[i].imshow(images[i])
    axes[i].axis('off')
    axes[i].set_title(labels[i])

# Plotting
plt.tight_layout()
plt.show()

### Validation generator from Validation folder (Dont need to proceed with Data Augmentation since its only for validation)

In [ ]:
# Validation generator from Validation folder (Dont need to proceed with Data Augmentation since its only for validation)
validation_generator = ImageDataGenerator().flow_from_directory (
                                                                 '/content/validation/',
                                                                 batch_size = 16,
                                                                 class_mode = 'categorical',
                                                                 target_size = (
                                                                                height,
                                                                                width
                                                                               )
                                                                )

### Inception V3 Model Call

In [ ]:
pretrainedModel = tf.keras.applications.InceptionV3(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrainedModel.trainable = False

### Train Model

In [ ]:
inputs = pretrainedModel.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrainedModel.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(5s, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

opt = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(
    optimizer= opt,
    loss='categorical_crossentropy',
    metrics=['accuracy','AUC']
)

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    batch_size = 16,
    epochs=5,
)
model.summary();

### Save Model

In [ ]:
model.save('/content/drive/MyDrive/Model/FinalModel.h5')

### Test Predict

In [ ]:
predicted_class = predict(model, '/content/alien/Mild/fe674c2f73f5.png')
print(predicted_class)